# 社群早期詐騙偵測演示

這個參考「[一年臺灣民眾被詐騙一千億，得覺醒](https://medium.com/@bohachu/%E4%B8%80%E5%B9%B4%E8%87%BA%E7%81%A3%E6%B0%91%E7%9C%BE%E8%A2%AB%E8%A9%90%E9%A8%99%E4%B8%80%E5%8D%83%E5%84%84-%E5%BE%97%E8%A6%BA%E9%86%92-11ba5b6ec18e)」文章的 Colab 檔案，模擬完整的端到端（End-to-end）詐騙偵測流程，包含：

*   **資料抓取**：
模擬從社群平台（如 Facebook、LINE）API 接收資料。
*   **圖-文多模態特徵工程**：
分別從成員結構（Graph）與貼文文字（Text）提取特徵。
*   **詐騙社群預測**：
結合圖與文字特徵，訓練一個 LightGBM 模型來預測詐騙分數。
*   **自動化處置**：
根據預測分數，執行相對應的降溫或標記策略。

In [ ]:
# @title 步驟一：環境設定與套件安裝

!pip install -q pandas numpy scikit-learn lightgbm networkx matplotlib seaborn
!pip install -q transformers sentence-transformers

print("✅ 套件安裝完成！")

In [ ]:
# @title 步驟二：模擬資料產生

import pandas as pd
import numpy as np
import networkx as nx

def generate_graph_features(is_scam=False):
    """ 產生圖徵（Graph Features） """
    if is_scam:
        # 詐騙社群：星狀擴散，一個核心節點連接大量新節點
        G = nx.star_graph(n=np.random.randint(50, 200))
        # 增加一些隨機邊來模擬複雜度
        for i in range(int(G.number_of_nodes() / 10)):
            u, v = np.random.choice(G.nodes(), 2, replace=False)
            G.add_edge(u, v)

        centrality = nx.degree_centrality(G)
        core_node = max(centrality, key=centrality.get)
        core_ness = centrality[core_node]
        burstiness = np.random.uniform(0.8, 1.0) # 爆發指數高
    else:
        # 正常社群：隨機網路，節點平均連接
        G = nx.erdos_renyi_graph(n=np.random.randint(50, 200), p=0.1)
        core_ness = np.mean(list(nx.degree_centrality(G).values()))
        burstiness = np.random.uniform(0.1, 0.4) # 爆發指數低

    return core_ness, burstiness

def generate_text_features(is_scam=False):
    """ 產生文字特徵（Text Features） """
    scam_keywords = ["高收益", "老師帶單", "保證獲利", "飆股", "內線消息"]
    normal_keywords = ["美食分享", "旅遊心得", "寵物日常", "電影推薦", "二手交換"]

    if is_scam:
        text = " ".join(np.random.choice(scam_keywords, 5, replace=True))
        hit_ratio = np.random.uniform(0.6, 1.0) # 關鍵字命中率高
    else:
        text = " ".join(np.random.choice(normal_keywords, 5, replace=True))
        hit_ratio = np.random.uniform(0.0, 0.2) # 關鍵字命中率低

    return text, hit_ratio

# --- 產生模擬資料 ---
data = []
for i in range(1000): # 產生 1000 筆社群資料
    is_scam = True if np.random.rand() > 0.7 else False # 30% 是詐騙社群

    core_ness, burstiness = generate_graph_features(is_scam)
    text, hit_ratio = generate_text_features(is_scam)

    data.append({
        "group_id": f"group_{i:04d}",
        "text": text,
        "core_ness": core_ness,
        "burstiness": burstiness,
        "keyword_hit_ratio": hit_ratio,
        "is_scam": 1 if is_scam else 0
    })

df = pd.DataFrame(data)

print("模擬資料集（前 5 筆）:")
print(df.head())
print("\n資料集維度:", df.shape)

In [ ]:
# @title 步驟三：文字特徵嵌入 (Text Embedding)

from sentence_transformers import SentenceTransformer

# 載入一個輕量的多語言模型作為範例
# 這邊我們用 sentence-transformers 內建模型來模擬 E5-moko 的效果
model_name = 'distiluse-base-multilingual-cased-v1'
text_embedder = SentenceTransformer(model_name)

print("正在將貼文文字轉換為向量...")
# 對所有貼文進行 embedding
text_embeddings = text_embedder.encode(df['text'].tolist(), show_progress_bar=True)

print("✅ 文字嵌入完成！")
print("嵌入向量維度:", text_embeddings.shape)

In [ ]:
# @title 步驟四：特徵整合與模型訓練

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 整合所有特徵
graph_features = df[['core_ness', 'burstiness', 'keyword_hit_ratio']].values
X = np.concatenate([graph_features, text_embeddings], axis=1)
y = df['is_scam'].values

# 2. 切分訓練集與測試集，並保留原始索引
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X, y, df.index, test_size=0.2, random_state=42, stratify=y
)


print(f"訓練集維度: {X_train.shape}")
print(f"測試集維度: {X_test.shape}")

# 3. 訓練 LightGBM 模型
lgb_classifier = lgb.LGBMClassifier(objective='binary', random_state=42)
lgb_classifier.fit(X_train, y_train)

print("\n✅ 模型訓練完成！")

# 4. 評估模型成效
y_pred_proba = lgb_classifier.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba > 0.5).astype(int) # 用 0.5 作為閥值

print("\n--- 模型評估報告 ---")
print(classification_report(y_test, y_pred))
print(f"AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")

# 繪製混淆矩陣
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# @title 步驟五：建立自動化處置策略

def automated_disposal_strategy(group_id, score):
    """
    根據詐騙分數執行自動化處置策略
    """
    print(f"--- 正在分析社群: {group_id} (詐騙分數: {score:.4f}) ---")

    if score > 0.9:
        print("🔴 高風險！執行處置...")
        # throttle_reach(): 降低觸及率 90%
        print("  - 執行 throttle_reach(): 觸及率 -90%")
        # flag_for_review(): 標記給人工審查
        print("  - 執行 flag_for_review(): 已提交人工審查")

    elif score > 0.8:
        print("🟡 中風險！執行處置...")
        # show_interstitial_warning_banner(): 顯示警告橫幅
        print("  - 執行 show_interstitial_warning_banner(): 已對使用者顯示警告橫幅")
    else:
        print("🟢 低風險。無需操作。")
    print("-" * 40)

# --- 模擬對新的社群資料進行即時偵測 ---
print("🚀 開始模擬即時偵測新進社群...\n")

# 從測試集中隨機抽取 5 個樣本來示範
sample_indices = np.random.choice(len(X_test), 5, replace=False)
sample_features = X_test[sample_indices]
# Use the stored test_indices to select from the original DataFrame
sample_info = df.loc[test_indices[sample_indices]]

# 使用訓練好的模型進行預測
predicted_scores = lgb_classifier.predict_proba(sample_features)[:, 1]

# 執行自動化策略
for i, (index, row) in enumerate(sample_info.iterrows()):
    group_id = row['group_id']
    score = predicted_scores[i]
    automated_disposal_strategy(group_id, score)

這個 Colab 檔案提供了一個從資料模擬到模型訓練，再到自動化策略應用的完整範例。您可以根據這個框架，替換成真實世界的數據和更複雜的模型，以應對實際的詐騙偵測挑戰。